# State Farm Distracted Driver Detection - Resnet50

This notebook contains the absolute final attempt at the Kaggle State Farm Distracted Driver Detection competition using the Resnet50 model. The purpose is to train the best possible model.

## Initial Setup

Import libraries and functions for future use.

In [1]:
# Plots displayed inline in notebook
%matplotlib inline

# Make help libraries available
import sys

sys.path.append('D:/anlaursen/libraries')

# Set visible devices, so as to just use a single GPU.
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import numpy as np
import pandas as pd
import gc

from kerastools.resnet50 import Resnet50
from kerastools.utils import get_batches, save_array, load_array, get_classes, do_clip

from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Dense, Dropout, Input
from keras.preprocessing import image

Using TensorFlow backend.


## Define model

We setup our initial Resnet50 model

In [3]:
resnet = Resnet50()
resnet.model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
lambda_1 (Lambda)                (None, 224, 224, 3)   0           input_1[0][0]                    
____________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D) (None, 230, 230, 3)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 112, 112, 64)  9472        zero_padding2d_1[0][0]           
___________________________________________________________________________________________

## Setup batches

We define out validation and training badges for modelling

In [4]:
batch_size = 32

#path = ''
path = 'sample/'

train_batches = resnet.get_batches(path + 'train', batch_size = batch_size)
val_batches = resnet.get_batches(path + 'valid', batch_size = batch_size, shuffle = False)

Found 1000 images belonging to 10 classes.
Found 100 images belonging to 10 classes.


## Finetune model - Sample

We need to adjust the standard VGG model to our new input with 10 classes, so we finetune it.

In [5]:
resnet.finetune(train_batches)
resnet.model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
lambda_1 (Lambda)                (None, 224, 224, 3)   0           input_1[0][0]                    
____________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D) (None, 230, 230, 3)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 112, 112, 64)  9472        zero_padding2d_1[0][0]           
___________________________________________________________________________________________

We train the model using the default learning rate of 0.001 for a single epoch

In [6]:
resnet.fit_batch(train_batches, val_batches, 1)

Epoch 1/1
31/31 [==============================] - 10s - loss: 2.2404 - acc: 0.1935 - val_loss: 2.2572 - val_acc: 0.2083


We see that the accuracy increases fine on the sample, so we increase the learning rate.

In [7]:
resnet.model.optimizer.lr = 0.1

resnet.fit_batch(train_batches, val_batches, 4)

Epoch 1/4
31/31 [==============================] - 8s - loss: 1.6417 - acc: 0.5050 - val_loss: 1.8361 - val_acc: 0.2500
Epoch 2/4
31/31 [==============================] - 6s - loss: 1.2824 - acc: 0.6744 - val_loss: 1.7445 - val_acc: 0.3235
Epoch 3/4
31/31 [==============================] - 6s - loss: 1.0734 - acc: 0.7573 - val_loss: 1.6593 - val_acc: 0.3529
Epoch 4/4
31/31 [==============================] - 6s - loss: 0.9168 - acc: 0.8047 - val_loss: 1.7541 - val_acc: 0.4265


Try 4 more epochs with lower learning rate.

In [8]:
resnet.model.optimizer.lr = 0.001

resnet.fit_batch(train_batches, val_batches, 4)

Epoch 1/4
31/31 [==============================] - 8s - loss: 0.7792 - acc: 0.8620 - val_loss: 1.6185 - val_acc: 0.4559
Epoch 2/4
31/31 [==============================] - 6s - loss: 0.6544 - acc: 0.9062 - val_loss: 1.4967 - val_acc: 0.4559
Epoch 3/4
31/31 [==============================] - 7s - loss: 0.5682 - acc: 0.9183 - val_loss: 1.5267 - val_acc: 0.4706
Epoch 4/4
31/31 [==============================] - 7s - loss: 0.5417 - acc: 0.9244 - val_loss: 1.4451 - val_acc: 0.3971


Seems, this is as far as we can get on the sample data set. A pretty good base line in the area of 0.5 - 0.66.

## Finetune model - Full data

We continue our finetuning on the full data set.

In [9]:
path = ''

train_batches = resnet.get_batches(path + 'train', batch_size = batch_size)
val_batches = resnet.get_batches(path + 'valid', batch_size = batch_size, shuffle = False)

Found 19624 images belonging to 10 classes.
Found 2800 images belonging to 10 classes.


We start with a single epoch

In [12]:
resnet.fit_batch(train_batches, val_batches, 1)

Epoch 1/1
613/613 [==============================] - 243s - loss: 0.4289 - acc: 0.9068 - val_loss: 1.2018 - val_acc: 0.6056


We see huge overfitting problems. We increase the learning rate and see, where that takes us.

In [13]:
resnet.model.optimizer.lr = 0.1

resnet.fit_batch(train_batches, val_batches, 4)

Epoch 1/4
613/613 [==============================] - 151s - loss: 0.2069 - acc: 0.9626 - val_loss: 1.2660 - val_acc: 0.5672
Epoch 2/4
613/613 [==============================] - 149s - loss: 0.1397 - acc: 0.9753 - val_loss: 1.3018 - val_acc: 0.5759
Epoch 3/4
613/613 [==============================] - 150s - loss: 0.1037 - acc: 0.9813 - val_loss: 1.2052 - val_acc: 0.6181
Epoch 4/4
613/613 [==============================] - 150s - loss: 0.0825 - acc: 0.9848 - val_loss: 1.2606 - val_acc: 0.6062


And then we lower the learning rate again, and see where we end up.

In [14]:
resnet.model.optimizer.lr = 0.001

resnet.fit_batch(train_batches, val_batches, 4)

Epoch 1/4
613/613 [==============================] - 153s - loss: 0.0655 - acc: 0.9881 - val_loss: 1.4437 - val_acc: 0.5762
Epoch 2/4
613/613 [==============================] - 151s - loss: 0.0535 - acc: 0.9908 - val_loss: 1.3009 - val_acc: 0.6109
Epoch 3/4
613/613 [==============================] - 153s - loss: 0.0461 - acc: 0.9917 - val_loss: 1.4294 - val_acc: 0.6001
Epoch 4/4
613/613 [==============================] - 152s - loss: 0.0391 - acc: 0.9935 - val_loss: 1.5603 - val_acc: 0.5762


So we get near perfect accuracy, but are overfitting quite a lot. Let's save the weights and try a different approach.

In [16]:
resnet.model.save_weights('models/base_resnet50.h5')

## Resnet precompute features
To experiment with different architectures, as well as doing knowledge destillation and pseudolabelling, we precompute the convolutional features for each layer. We start by defining a new model with no dense layers.

In [3]:
resnet_conv = Resnet50(include_top = False)

Let's check out the model

In [4]:
resnet_conv.model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
lambda_1 (Lambda)                (None, 224, 224, 3)   0           input_1[0][0]                    
____________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D) (None, 230, 230, 3)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 112, 112, 64)  9472        zero_padding2d_1[0][0]           
___________________________________________________________________________________________

Define new batches with proper dimensions, such as to get the right number of pre computed convolutional terms .

In [5]:
path = ''

train_batches = get_batches(path + 'train', batch_size = 44, target_size = (224, 224), shuffle = False)

valid_batches = get_batches(path + 'valid', batch_size = 50, target_size = (224, 224), shuffle = False)

test_batches = get_batches(path + 'test', batch_size = 2, target_size = (224, 224), shuffle = False, class_mode = None)

Found 19624 images belonging to 10 classes.
Found 2800 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


We also extract labels and classes for each dataset.

In [6]:
(val_classes, trn_classes, val_labels, trn_labels, 
 val_filenames, filenames, test_filenames) = get_classes(path)

Found 19624 images belonging to 10 classes.
Found 2800 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


We then pre-compute each of our datasets and save the numpy arrays. This eats a lot of memory on the poor AWS instance, so after each data computation, we do some cleanup to realease the memory. We save and load using bcolz, as it utilises great compression and does I/O very fast.

In [11]:
conv_feat = resnet_conv.model.predict_generator(train_batches, np.int(train_batches.samples / train_batches.batch_size))
save_array(path + 'results/conv_computed/conv_feat_resnet.dat', conv_feat)

del conv_feat
gc.collect()

20

In [12]:
conv_val_feat = resnet_conv.model.predict_generator(valid_batches, np.int(valid_batches.samples / valid_batches.batch_size))
save_array(path + 'results/conv_computed/conv_val_feat_resnet.dat', conv_val_feat)

del conv_val_feat
gc.collect()

0

In [13]:
conv_test_feat = resnet_conv.model.predict_generator(test_batches, np.int(test_batches.samples / test_batches.batch_size))
save_array(path + 'results/conv_computed/conv_test_feat_resnet.dat', conv_test_feat)

del conv_test_feat
gc.collect()

0

And finally, we can load the three feature sets

In [7]:
conv_feat = load_array('results/conv_computed/conv_feat_resnet.dat')
conv_val_feat = load_array('results/conv_computed/conv_val_feat_resnet.dat')
conv_test_feat = load_array('results/conv_computed/conv_test_feat_resnet.dat')

## VGG model with precomputed augmentation

Precompute a bunch of random images

In [7]:
gen_t = image.ImageDataGenerator(rotation_range = 15,
                                 height_shift_range = 0.05,
                                 shear_range = 0.1,
                                 channel_shift_range = 20,
                                 width_shift_range = 0.1)

path = ''
da_batches = get_batches(path + 'train',
                            gen_t,
                            batch_size = 11,
                            shuffle = True,
                            target_size = (224, 224))
val_batches = get_batches(path + 'valid', batch_size = 25, shuffle = False)

Found 19624 images belonging to 10 classes.
Found 2800 images belonging to 10 classes.


We then make 10 full cycles (epochs) of the training data, precomputing the convolutional layer on the augmented images.

In [8]:
da_conv_feat = resnet_conv.model.predict_generator(da_batches, np.int(da_batches.samples / da_batches.batch_size) * 10)
save_array(path + 'results/conv_computed/da_conv_feat_resnet.dat', da_conv_feat)

del da_conv_feat
gc.collect()

InternalError: Dst tensor is not initialized.
	 [[Node: activation_49/Relu/_1383 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_4501_activation_49/Relu", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

We can then load the pre-compuited and augmented data.

In [ ]:
da_conv_feat = load_array(path + 'results/conv_computed/da_conv_feat_resnet.dat')

And then concatenate it with our existing precomputed training data.

In [ ]:
da_conv_feat = np.concatenate([da_conv_feat, conv_feat])

We now have 11 times as many features, but luckily order is preserved, so we can just multiply the labels with 11.

In [ ]:
da_trn_labels = np.concatenate([trn_labels] * 11)

We can then define a dense layer model.

In [ ]:
def get_dense_resnet(base_model, p):
    
    inputs = Input(shape = base_model.layers[-1].output_shape[1:])
        
    dense_layer = Dropout(p)(inputs
    Dense(10, activation='softmax')(dense_layer)                         
                             
    dense_layer = Dense(10, activation='softmax')(dense_layer)
    
    model = Model(inputs = inputs, outputs = dense_layer)
    
    return model
    
# Define level of dropout
p = 0.2

# Define batch size
batch_size = 32

Get the dense model.

In [58]:
dense_resnet = get_dense_resnet(resnet_conv.model, p)

dense_resnet.compile(optimizer = Adam(lr = 0.0001),
                     loss = 'categorical_crossentropy',
                     metrics = ['accuracy'])

dense_resnet.fit(x = da_conv_feat,
                 y = da_trn_labels,
                 batch_size = batch_size,
                 epochs = 1,
                 validation_data = (conv_val_feat, val_labels))

Train on 215864 samples, validate on 2800 samples
Epoch 1/1
215864/215864 [==============================] - 28s - loss: 2.4821 - acc: 0.1156 - val_loss: 2.0634 - val_acc: 0.2936


Then train the model using the augmented batches. First run some epochs at default learning rate.

In [59]:
dense_resnet.fit(x = da_conv_feat,
                 y = da_trn_labels,
                 batch_size = batch_size,
                 epochs = 4,
                 validation_data = (conv_val_feat, val_labels))

Train on 215864 samples, validate on 2800 samples
Epoch 1/4
215864/215864 [==============================] - 27s - loss: 2.3486 - acc: 0.1393 - val_loss: 1.9997 - val_acc: 0.3496
Epoch 2/4
215864/215864 [==============================] - 27s - loss: 2.3059 - acc: 0.1540 - val_loss: 1.9713 - val_acc: 0.3921
Epoch 3/4
215864/215864 [==============================] - 27s - loss: 2.2878 - acc: 0.1605 - val_loss: 1.9615 - val_acc: 0.3989
Epoch 4/4
215864/215864 [==============================] - 27s - loss: 2.2782 - acc: 0.1651 - val_loss: 1.9480 - val_acc: 0.4275


In [60]:
dense_resnet.fit(x = da_conv_feat,
                 y = da_trn_labels,
                 batch_size = batch_size,
                 epochs = 20,
                 validation_data = (conv_val_feat, val_labels))

Train on 215864 samples, validate on 2800 samples
Epoch 1/20
215864/215864 [==============================] - 27s - loss: 2.2737 - acc: 0.1673 - val_loss: 1.9340 - val_acc: 0.4721
Epoch 2/20
215864/215864 [==============================] - 26s - loss: 2.2704 - acc: 0.1696 - val_loss: 1.9496 - val_acc: 0.4207
Epoch 3/20
215864/215864 [==============================] - 26s - loss: 2.2674 - acc: 0.1707 - val_loss: 1.9255 - val_acc: 0.4614
Epoch 4/20
215864/215864 [==============================] - 26s - loss: 2.2674 - acc: 0.1710 - val_loss: 1.9128 - val_acc: 0.4907
Epoch 5/20
215864/215864 [==============================] - 26s - loss: 2.2663 - acc: 0.1712 - val_loss: 1.9097 - val_acc: 0.4846
Epoch 6/20
215864/215864 [==============================] - 26s - loss: 2.2655 - acc: 0.1718 - val_loss: 1.9220 - val_acc: 0.4718
Epoch 7/20
215864/215864 [==============================] - 28s - loss: 2.2647 - acc: 0.1715 - val_loss: 1.9165 - val_acc: 0.4646
Epoch 8/20
215864/215864 [==============

Then increase the learning rate and run some more epochs

In [61]:
dense_resnet.optimizer.lr = 0.001

dense_resnet.fit(x = da_conv_feat,
                 y = da_trn_labels,
                 batch_size = batch_size,
                 epochs = 20,
                 validation_data = (conv_val_feat, val_labels))

Train on 215864 samples, validate on 2800 samples
Epoch 1/20
215864/215864 [==============================] - 27s - loss: 2.2636 - acc: 0.1735 - val_loss: 1.9052 - val_acc: 0.4489
Epoch 2/20
215864/215864 [==============================] - 27s - loss: 2.2640 - acc: 0.1735 - val_loss: 1.8963 - val_acc: 0.4729
Epoch 3/20
215864/215864 [==============================] - 27s - loss: 2.2645 - acc: 0.1729 - val_loss: 1.8984 - val_acc: 0.4611
Epoch 4/20
215864/215864 [==============================] - 27s - loss: 2.2644 - acc: 0.1735 - val_loss: 1.9063 - val_acc: 0.4514
Epoch 5/20
215864/215864 [==============================] - 28s - loss: 2.2635 - acc: 0.1741 - val_loss: 1.8923 - val_acc: 0.4775
Epoch 6/20
215864/215864 [==============================] - 27s - loss: 2.2640 - acc: 0.1736 - val_loss: 1.8867 - val_acc: 0.4850
Epoch 7/20
215864/215864 [==============================] - 27s - loss: 2.2641 - acc: 0.1736 - val_loss: 1.8868 - val_acc: 0.5061
Epoch 8/20
215864/215864 [==============

KeyboardInterrupt: 

Finally decrease the learning rate and run some more epochs.

In [32]:
resnet.model.optimizer.lr = 0.001

resnet.fit_batch(train_batches, val_batches, 20)

Epoch 1/20
446/446 [==============================] - 407s - loss: 0.1860 - acc: 0.9416 - val_loss: 1.4871 - val_acc: 0.6326
Epoch 2/20
446/446 [==============================] - 363s - loss: 0.1858 - acc: 0.9393 - val_loss: 1.3615 - val_acc: 0.6192
Epoch 3/20
446/446 [==============================] - 358s - loss: 0.1945 - acc: 0.9381 - val_loss: 1.4962 - val_acc: 0.5979
Epoch 4/20
446/446 [==============================] - 357s - loss: 0.1859 - acc: 0.9419 - val_loss: 1.2958 - val_acc: 0.6308
Epoch 5/20
446/446 [==============================] - 363s - loss: 0.1792 - acc: 0.9433 - val_loss: 1.3871 - val_acc: 0.6171
Epoch 6/20
446/446 [==============================] - 366s - loss: 0.1852 - acc: 0.9405 - val_loss: 1.1916 - val_acc: 0.6488
Epoch 7/20
446/446 [==============================] - 369s - loss: 0.1824 - acc: 0.9429 - val_loss: 1.5571 - val_acc: 0.5762
Epoch 8/20
446/446 [==============================] - 377s - loss: 0.1748 - acc: 0.9429 - val_loss: 1.3423 - val_acc: 0.6337


## Pseudolabeling

We're going to try using a combination of [pseudo labeling](http://deeplearning.net/wp-content/uploads/2013/03/pseudo_label_final.pdf) and [knowledge distillation](https://arxiv.org/abs/1503.02531) to allow us to use unlabeled data (i.e. do semi-supervised learning). For our initial experiment we'll use the validation set as the unlabeled data, so that we can see that it is working without using the test set. Afterwards we add the test set as well.

In [22]:
val_pseudo = resnet.predict(val_batches, batch_size = 50)

We concatenate thse pseudo labels with our training labels

In [24]:
comb_pseudo = np.concatenate([da_trn_labels, val_pseudo])
comb_feat = np.concatenate([da_conv_feat, conv_val_feat])

And train our model using the extended data set.